In [ ]:
import numpy as np
import datetime
import pickle
from matplotlib import pyplot as plt

from utils.data_reading.sound_data.sound_file_manager import make_manager
from utils.physics.signal.make_spectrogram import make_spectrogram

In [ ]:
file = "../../../data/detection/TiSSNet/MAHY0_res/MAHY1"
sound_file_path = "/media/plerolland/akoustik/MAHY/2020/MAHY1"

MIN_P = 0.3

manager = make_manager(sound_file_path)

DELTA = datetime.timedelta(seconds=100)
TIME_RES = 0.5
FREQ_RES = 0.5

dets = []
with open(file, "rb") as f:
    while True:
        try:
            dets.append(pickle.load(f))
        except EOFError:
            break
dets = np.array(dets)
print(f"Found {len(dets)} detections")

In [ ]:
N = 5

fig, axs = plt.subplots(N, 1, figsize=(15, 7*N))

dets = dets[dets[:,1] > MIN_P]
np.random.shuffle(dets)
dets_displayed = dets[:N]
#dets_displayed = dets[np.argsort(dets[:,1])[::-1]][:N]

for i, (det_date, det_p) in enumerate(dets_displayed):
    data = manager.get_segment(det_date-DELTA, det_date+DELTA)
    f, t, spectro = make_spectrogram(data, fs=manager.sampling_f, f_res=FREQ_RES, t_res=TIME_RES, return_bins=True, normalize=True)

    axs[i].imshow(spectro, extent=(t[0], t[-1], f[0], f[-1]), cmap="jet", aspect="auto")
    axs[i].set_title(f"{det_p} at {det_date}")